In [1]:
# libraries importing
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from sklearn.preprocessing import MinMaxScaler
import numpy as np

from tensorflow import keras
from keras.models import Model, load_model
from keras.layers import Input, Dense, LSTM, GRU, RepeatVector, TimeDistributed
from keras.callbacks import EarlyStopping

from tqdm import tqdm 

c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\_distributor_init.py:30: UserWarning: loaded more than 1 DLL from .libs:
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.EL2C6PLE4ZYW3ECEVIV3OXXGRN2NRFM2.gfortran-win_amd64.dll
c:\Users\gioel\AppData\Local\Programs\Python\Python310\lib\site-packages\numpy\.libs\libopenblas.FB5AE2TYXYH2IJRDKGDGQ3XBKLKTF43H.gfortran-win_amd64.dll
  warnings.warn("loaded more than 1 DLL from .libs:"


In [2]:
WINDOW_SIZE=40
def create_sequences(values, time_steps=WINDOW_SIZE):
    output = []
    for i in range(len(values) - time_steps + 1):
        output.append(values[i : (i + time_steps)])
    return np.stack(output)

    
def MODEL():
    inputs = Input(shape=(WINDOW_SIZE, 19))
    z = LSTM(128, return_sequences=True)(inputs)
    z = LSTM(64)(z)
    z = RepeatVector(WINDOW_SIZE)(z)
    z = LSTM(64, return_sequences=True)(z)
    z = LSTM(128, return_sequences=True)(z)
    outputs = TimeDistributed(Dense(19))(z)

    model = Model(inputs, outputs, name="LSTM-AE")
    model.compile(loss="mse", optimizer=keras.optimizers.Adam(), metrics=["mse"])

    #model.summary()
    return model

In [3]:
import pickle
with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
            DATA = pickle.load(f)
    
model=MODEL()

model.load_weights(filepath=f"MODEL_AE/model_apptot_Win_40_.h5")

e=model.predict(DATA['X_test'])
score=np.mean(np.power(e-DATA['X_test'],2),axis=(1,2))

np.save('./score_analisi/LSTM_AE.npy',score,allow_pickle=True)


SAVE result

In [5]:
import csv
import pickle

header=['th_factor','method','value','F1','precision','recall','TP','TN','FP','FN']

with open('LSTM_AE.csv', 'w', encoding='UTF8') as f:
    writer = csv.writer(f)

    writer.writerow(header)
    thresholding_factor=[0.5,1,1.5,2]
    for t in thresholding_factor:
        with open(f'./DATA_SPLITTED/app_tot.pkl', 'rb') as f:
            DATA= pickle.load(f)
        model= MODEL()
        
        model.load_weights(filepath=f'MODEL_AE/model_apptot_Win_40_.h5')


        e=model.predict(DATA['X_test'])
        score=np.mean(np.power(e-DATA['X_test'],2),axis=(1,2))

        

        #IQR
        q1, q3 = np.percentile(score, 25), np.percentile(score, 75)
        iqr = q3 - q1
        IQR = q3 + t* iqr

        #MAD
        median = np.median(score)
        mad = 1.4826 * np.median(np.abs(score - median))
        MAD = median + t * mad

        #STD
        mean, std = np.mean(score), np.std(score)
        STD = mean + t * std

        method=[IQR,MAD,STD]
        
        for g in range(len(method)):
            TP=0
            TN=0
            FN=0
            FP=0

            for i in tqdm(range(1,11)):
                if(i!=7):
                
                            TEST=np.load(f'./OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/test{i}.npy',allow_pickle=True)

                            ANOMALY=np.load(f'./OUTPUTS_ROOT/data/processed/spark_0_15s/spark_0_trace-scl_std/y_test{i}.npy',allow_pickle=True)

                            with open(f'./OUTPUTS_ROOT/data/interim/spark_0_15s/test_info{i}.pkl', 'rb') as f:
                                TEST_info= pickle.load(f)

                            
                            for x in range(len(TEST_info)):

                                X=create_sequences(TEST[x])

                                Y=model.predict(X)
                                error=np.mean(np.power(X-Y,2),axis=(1,2))

                                error=[error[l] if error[l]<2 else 2 for l in range(len(error))]

                                outlier=error>method[g]

                                lunghezza_minima_errore=20
                                length=0
                                for h in range(len(outlier)):
                                    if(outlier[h]==True):
                                        if(length<lunghezza_minima_errore):
                                            for g in range(length+1):
                                                outlier[h-g]=True
                                        length=0
                                    else:
                                        length=length+1

                                lunghezza_minima_errore=40
                                length=0
                                for h in range(len(outlier)):
                                    if(outlier[h]==True):
                                        length=length+1
                                    else:
                                        if(length<lunghezza_minima_errore):
                                            for g in range(length+1):
                                                outlier[h-g]=False
                                        length=0
                                le=len(outlier)

                                true_= ANOMALY[x][20:le+20]>=1
                                prediction_ = outlier[:].astype(int)==1
                            
                                TP = TP+(true_ & prediction_).sum()   
                                TN = TN+(~true_ & ~prediction_).sum()  
                                FP = FP+(~true_ & prediction_).sum()    
                                FN = FN+(true_ & ~prediction_).sum()        

            PREC=TP / (TP + FP)
            REC = TP/ (TP+FN)
            f1=2 * PREC * REC/(PREC + REC)
            m=['IQR','MAD','STD']
            row=[t,m[g],method[g],f1,PREC,REC,TP,TN,FP,FN]
            writer.writerow(row)

100%|██████████| 10/10 [01:30<00:00,  9.10s/it]
